In [5]:
# -*- coding: utf-8 -*-
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import joblib
import scipy.integrate as integrate
from __future__ import unicode_literals
from sklearn import preprocessing as prep
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.mixture import GaussianMixture as gm
from multiprocessing import Pool
from sklearn.cluster import KMeans

In [6]:
class LOB:
    def __init__(self, wiersz):
        self.zmiana = 0
        self.dzien = wiersz[0]
        self.czas = wiersz[1]
        i = 3
        while (wiersz[i] != 'ASK'):
            i += 1
        bid = wiersz[3:i]
        ask = wiersz[(i+1):]
        self.bid = np.zeros((int((len(bid) / 2)), 2))
        self.ask = np.zeros((int((len(ask) / 2)), 2))
        for i in range(0, len(bid), 2):
            self.bid[int(i / 2)][0] = float(bid[i]) 
            self.bid[int(i / 2)][1] = float(bid[i+1])
        for i in range(0, len(ask), 2):
            self.ask[int(i / 2)] = [float(ask[i]), float(ask[i+1])]
        self.midPrice = (self.bid[len(self.bid) - 1][0] + self.ask[0][0]) / 2
    def __str__(self):
        return "Dzień: " + self.dzien + ", Czas: " + self.czas + ", midPrice: " + str(self.midPrice)
    
X = np.load("X.npy")
X1 = np.load("X1.npy")
X2 = np.load("X2.npy")
X3 = np.load("X3.npy")
X4 = np.load("X4.npy")
zmiana = np.load("zmiana.npy")
Imbalance1 = np.load("Imbalance1.npy")
indeksy = np.load("indeksy.npy")
tablica = np.load("tablica.npy")
gmmList = np.load("gmmList.npy")
tablicaCalekPo1 = np.load("tablCalekPo1.npy")
tablicaCalekPo1BezDzielenia = np.load("tabCalekPo1BezDz.npy")
tablicaTickowPo1 = np.load("tablicaTickowPo1.npy")
tablicaTickowRosnaca = np.load("tablicaTickowRosnaca.npy")
tablicaCalekRosnaca = np.load("tablicaCalekRosnaca.npy")
tablicaCalekRosnacaBezDzielenia = np.load("tablicaCalekRosnacaBezDzielenia.npy")

def normalizuj(tab):
    tabMean = np.mean(tab, axis = 0)
    tabStd = np.std(tab, axis = 0)
    tabStd[(tabStd == 0)] = 1
    return ((tab - tabMean) / tabStd)

def calkaBezDzielenia(a, b, gmm, eps):
    x = np.arange(a, b + 0.0000001, eps)
    y = np.exp(gmm.score_samples(x.reshape(-1, 1)))
    return np.sum(y[:x.shape[0] - 1] + y[1:])*eps

def calka(a, b, gmm, eps):
    n = (int) ((b - a) / eps)
    return (calkaBezDzielenia(a, b, gmm, eps) / 2)

def obieCalki(a, b, gmm, eps):
    n = (int) ((b - a) / eps)
    wynik = calkaBezDzielenia(a, b, gmm, eps)*(b - a)
    return wynik, wynik/n/2

def calkaBezDzieleniaF(a, b, f, eps):
    x = np.arange(a, b + 0.0000001, eps)
    y = f(x.reshape(-1, 1))
    return np.sum(y[:x.shape[0] - 1] + y[1:])*eps

def calkaF(a, b, gmm, eps):
    n = (int) ((b - a) / eps)
    return (calkaBezDzieleniaF(a, b, gmm, eps) / 2)
    
# ustawianie wartości zmian i tworzenie dla nich osobnego wektora
def generujZmiane(X):
    for i in range(len(X) - 1):
        if (X[i].midPrice > X[i + 1].midPrice):
            X[i].zmiana = 1
        if (X[i].midPrice < X[i + 1].midPrice):
            X[i].zmiana = -1
        if (X[i].midPrice == X[i + 1].midPrice):
            X[i].zmiana = 0
    return np.array([i.zmiana for i in X])

# interesują nas te loby, po którch nastąpiła zmiana mid price
def generujIndeksy(X):
    return np.arange(len(X))[np.array([(i.zmiana != 0) for i in X])]

# pierwszy sposób - imbalance to różnica najlepszego bid'a i ask'a przez ich sumę
def generujImbalance(X):
    i = 0
    Imbalance = np.zeros((X.shape[0], 1))
    for wiersz in X: 
        bid = wiersz.bid[-1, 1]
        ask = wiersz.ask[0, 1]
        Imbalance[i] = (bid - ask) / (bid + ask)
        i += 1
    return Imbalance

def generujWektor(k):
    dlugosc = imbalance.shape[0]
    ImbalanceWektor = np.zeros((dlugosc, k))
    for i in range(k, dlugosc):
        ImbalanceWektor[i] = imbalance[(i-k) : i].reshape(k)
    return ImbalanceWektor

def testujDecisionFunction(xTest, yTest, najDecFun, clf):
    wynik = clf.decision_function(xTest).reshape(yTest.shape)
    yTestWiekszeOdZera = yTest > 0
    if (najDecFun[1]):
        return np.sum((wynik > najDecFun[0]) == yTestWiekszeOdZera) / yTest.shape[0]
    return np.sum((wynik < najDecFun[0]) == yTestWiekszeOdZera) / yTest.shape[0]

def najlepszeDecFun(yTrain, wynikDecFun):
    yTrainWiekszeOdZera = yTrain > 0
    tabWynikow = []
    for i in range(wynikDecFun.shape[0]):
        wy = np.sum((wynikDecFun < wynikDecFun[i]) == yTrainWiekszeOdZera) / yTrain.shape[0]
        tabWynikow.append(wy)
        wy = np.sum((wynikDecFun > wynikDecFun[i]) == yTrainWiekszeOdZera) / yTrain.shape[0]
        tabWynikow.append(wy)
    pozycjaMaxArg = np.argmax(tabWynikow)
    if (pozycjaMaxArg % 2 == 0):
        return (wynikDecFun[int(pozycjaMaxArg / 2)], 0)
    else:
        return (wynikDecFun[int(pozycjaMaxArg / 2)], 1)

def crossValidation(clf, data, dataClass, k):
    size = data.shape[0]
    arr = np.arange(size)
    np.random.shuffle(arr)
    err = 0
    n = int(size / k)
    for i in range(k):
        mask2 = arr[np.arange(i*n,min((i+1)*n,size))]
        mask1 = arr[np.concatenate((np.arange(i*n),np.arange((i+1)*n,size)))]
        X_train = data[mask1]
        y_train = dataClass[mask1]
        X_test = data[mask2]
        y_test = dataClass[mask2]  
        clf = clf.fit(X_train, y_train)
        if isinstance(clf, GridSearchCV):
            clf = clf.best_estimator_
        y_train = y_train.reshape((y_train.shape[0], 1))
        wynikDecFun = clf.decision_function(X_train).reshape(y_train.shape)
        najDecFun = najlepszeDecFun(y_train, wynikDecFun)        
        err += testujDecisionFunction(X_test, y_test, najDecFun, clf)
    return float(err) / k

def generujPozycje(midPrice):
    pozMidPrice = (midPrice - minimum) * 10
    pozMidPriceInt = int(pozMidPrice)
    pozLewa = pozMidPriceInt
    pozPrawa = pozMidPriceInt + 1
    if (pozMidPrice == pozMidPriceInt):
        pozLewa -= 1
    return (pozLewa, pozPrawa)

def generujPrzedzialy(midPrice, ilosc, f):
    pozLewa, pozPrawa = generujPozycje(midPrice)
    przedzialy = []
    wspXowe = []
    for i in range(ilosc):
        fi = f(i)
        wspXowe.append((pozPrawa / 10) + minimum + (fi / 2))
        wspXowe.append((pozLewa / 10) + minimum - (fi / 2))
        fi10 = int(fi*10)
        tmpList = np.arange(fi10)
        przedzialy.append(pozPrawa + tmpList)
        pozPrawa += fi10
        pozLewa -= fi10
        przedzialy.append(pozLewa + 1 + tmpList)
    return przedzialy, wspXowe
        
def generujTabliceTickow(ilosc, f):
    wynik = np.zeros((len(X), 2 * ilosc))
    listaWspXowych = []
    for i in range(len(X)):
        wiersz = tablica[i]
        przedzialy, wspXowe = generujPrzedzialy(X[i].midPrice, ilosc, f)
        listaWspXowych.append(wspXowe)
        for j in range(2 * ilosc):
            przedzial = przedzialy[j]
            przedzial = przedzial[przedzial >= 0]
            przedzial = przedzial[przedzial < liczbaWspolrzednych]
            wynik[i][j] = np.sum(wiersz[przedzial])
    return wynik, listaWspXowych

def generujTabliceTickowDlaGMM(ilosc, f):
    wynik = np.zeros((len(X), 2 * ilosc))
    listaWspXowych = []
    for i in range(len(X)):
        wiersz = tablica[i]
        pozLewa = X[i].midPrice
        pozPrawa = pozLewa
        
        for j in range(ilosc):
            fj = f(j)
            wynik[i][2*j] = calka(pozPrawa, pozPrawa + fj, gmmList[i], 0.001)
            wynik[i][2*j + 1] = calka(pozLewa - fj, pozLewa, gmmList[i], 0.001)
            pozPrawa += fj
            pozLewa -= fj            
    return wynik

def wyliczCViWypisz(nazwa, tablica, mnoznik, liczbaKolumn, iloscIteracji):
    if mnoznik != 1:
        nazwa += '*' + str(mnoznik)
    wynik = crossValidation(svm, (((tablica[oIndeksy]).T[:liczbaKolumn]).T)*mnoznik, zmiana, iloscIteracji)
    print(nazwa, "zredukowana do ", liczbaKolumn, " kolumn, Cross-Vall score: ", wynik)
    
def funkcjaPrzedzialow(i):
    return (int((0.1 + (0.05 + i / 20) * np.sqrt(i)) * 10) / 10)
    
minimum = 100000
maksimum = 0
for wiersz in X:
    bid = wiersz.bid[0, 0]
    ask = wiersz.ask[-1, 0]
    if(ask > maksimum):
        maksimum = ask
    if(bid < minimum):
        minimum = bid 
liczbaWspolrzednych = int((maksimum - minimum) * 10 + 1)

def testujDecisionFunction(xTest, yTest, najDecFun, clf):
    wynik = clf.decision_function(xTest).reshape(yTest.shape)
    yTestWiekszeOdZera = yTest > 0
    if (najDecFun[1]):
        return np.sum((wynik > najDecFun[0]) == yTestWiekszeOdZera) / yTest.shape[0]
    return np.sum((wynik < najDecFun[0]) == yTestWiekszeOdZera) / yTest.shape[0]

def najlepszeDecFun(yTrain, wynikDecFun):
    yTrainWiekszeOdZera = yTrain > 0
    tabWynikow = []
    for i in range(wynikDecFun.shape[0]):
        wy = np.sum((wynikDecFun < wynikDecFun[i]) == yTrainWiekszeOdZera) / yTrain.shape[0]
        tabWynikow.append(wy)
        wy = np.sum((wynikDecFun > wynikDecFun[i]) == yTrainWiekszeOdZera) / yTrain.shape[0]
        tabWynikow.append(wy)
    pozycjaMaxArg = np.argmax(tabWynikow)
    if (pozycjaMaxArg % 2 == 0):
        return (wynikDecFun[int(pozycjaMaxArg / 2)], 0)
    else:
        return (wynikDecFun[int(pozycjaMaxArg / 2)], 1)

def crossValidation(clf, data, dataClass, k):
    size = data.shape[0]
    arr = np.arange(size)
    np.random.shuffle(arr)
    err = 0
    n = int(size / k)
    for i in range(k):
        mask2 = arr[np.arange(i*n,min((i+1)*n,size))]
        mask1 = arr[np.concatenate((np.arange(i*n),np.arange((i+1)*n,size)))]
        X_train = data[mask1]
        y_train = dataClass[mask1]
        X_test = data[mask2]
        y_test = dataClass[mask2]  
        clf = clf.fit(X_train, y_train)
        y_train = y_train.reshape((y_train.shape[0], 1))
        wynikDecFun = clf.decision_function(X_train).reshape(y_train.shape)
        najDecFun = najlepszeDecFun(y_train, wynikDecFun)        
        err += testujDecisionFunction(X_test, y_test, najDecFun, clf)
    return float(err) / k

def crossValidationPoKolei(clf, data, dataClass, k, p):
    size = data.shape[0]
    arr = np.arange(size)
    err = 0
    n = int(size / (k+1))
    trainCount = int(2 * n * p)
    for i in range(k):
        trainMask = np.arange(i*n, i*n + trainCount)
        testMask = np.arange(i*n + trainCount, (i+2)*n)
        X_train = data[trainMask]
        y_train = dataClass[trainMask]
        X_test = data[testMask]
        y_test = dataClass[testMask]  
        clf = clf.fit(X_train, y_train)
        y_train = y_train.reshape((y_train.shape[0], 1))
        wynikDecFun = clf.decision_function(X_train).reshape(y_train.shape)
        najDecFun = najlepszeDecFun(y_train, wynikDecFun)        
        wynik = testujDecisionFunction(X_test, y_test, najDecFun, clf)
        err += wynik
    return float(err) / k

def czas(h, m):
    mnoznikM = 100000
    mnoznikH = 10000000
    return h*mnoznikH + m*mnoznikM

def czasOdwrotny(x):
    mnoznikM = 100000
    mnoznikH = 10000000
    return (int(x / mnoznikH), int((x % mnoznikH) / mnoznikM))

for i in range (len(X)):
    X[i].czas = int(X[i].czas)
    X[i].dzien = int(X[i].dzien)
    
# wszystko było na przełomie 2013.09, więc obchodzi nas tylko dzień
for i in range (len(X)):
    X[i].dzien = X[i].dzien % 100
    
zmiana = generujZmiane(X)
maskaCzasu = np.array([wiersz.czas > czas(8, 0) and wiersz.czas < czas(16, 45) for wiersz in X])
oX = X[maskaCzasu]
indeksy = generujIndeksy(X)
oIndeksy = []
for indeks in indeksy:
    if (maskaCzasu[indeks]):
        oIndeksy.append(indeks)
oIndeksy = np.array(oIndeksy)
imbalance = generujImbalance(X)

In [7]:
def test(indeksy, nazwa, tablica, tablica2, tablica3, svm, ilosc = 4, procent = 0.75, iloscKolumn = 120):
    wynik1 = crossValidationPoKolei(svm, (((tablica[indeksy]).T[:iloscKolumn]).T), zmiana[indeksy], ilosc, procent)
    wynik2 = crossValidationPoKolei(svm, (((tablica2[indeksy]).T[:iloscKolumn]).T), zmiana[indeksy], ilosc, procent)
    wynik3 = crossValidationPoKolei(svm, (((tablica3[indeksy]).T[:iloscKolumn]).T), zmiana[indeksy], ilosc, procent) 
    print("Czyste dane:", wynik1, ", S+N:", wynik2, ", N+S:", wynik3)
    
def test2(indeksy, nazwa, tablica, tablica2, tablica3, tablica4, tablica5, svm, ilosc = 4, procent = 0.75, iloscKolumn = 120):
    wynik1 = crossValidationPoKolei(svm, (((tablica[indeksy]).T[:iloscKolumn]).T), zmiana[indeksy], ilosc, procent)
    wynik2 = crossValidationPoKolei(svm, (((tablica2[indeksy]).T[:iloscKolumn]).T), zmiana[indeksy], ilosc, procent)
    wynik3 = crossValidationPoKolei(svm, (((tablica3[indeksy]).T[:iloscKolumn]).T), zmiana[indeksy], ilosc, procent)
    wynik4= crossValidationPoKolei(svm, (((tablica4[indeksy]).T[:iloscKolumn]).T), zmiana[indeksy], ilosc, procent) 
    wynik5 = crossValidationPoKolei(svm, (((tablica5[indeksy]).T[:iloscKolumn]).T), zmiana[indeksy], ilosc, procent) 

    return wynik1, wynik2, wynik3, wynik4, wynik5

def zwrocDzien(x):
    return {
        0 : 'poniedziałek',
        1 : 'wtorek',
        2 : 'środa',
        3 : 'czwartek',
        4 : 'piątek'
    }[x % 5]

In [8]:
def crossValidationPrzedzialy(clf, data, listaIndeksow, p):
    dataClass = zmiana
    err = 0
    i = 0
    listaWynikow = []
    for indeksy in listaIndeksow:
        n = indeksy.shape[0]
        trainCount = int(p*n)
        trainMask = indeksy[np.arange(trainCount)]
        testMask = indeksy[np.arange(trainCount, n)]
        X_train = data[trainMask]
        y_train = dataClass[trainMask]
        X_test = data[testMask]
        y_test = dataClass[testMask]  
        clf = clf.fit(X_train, y_train)
        y_train = y_train.reshape((y_train.shape[0], 1))
        wynikDecFun = clf.decision_function(X_train).reshape(y_train.shape)
        najDecFun = najlepszeDecFun(y_train, wynikDecFun)        
        wynik = testujDecisionFunction(X_test, y_test, najDecFun, clf)
        listaWynikow.append(wynik)
        err += wynik
        i += 1
    return float(err) / len(listaIndeksow), listaWynikow

def wyliczCVPoKoleiiWypisz(indeksy, nazwa, tablica, iloscIteracji, p, liczbaKolumn, svm):
    wynik = crossValidationPoKolei(svm, (((tablica[indeksy]).T[:liczbaKolumn]).T), zmiana[indeksy], iloscIteracji, p)
    print(nazwa, "zredukowana do ", liczbaKolumn, " kolumn, Cross-Vall score: ", wynik)
    print()
    
param_grid = {'C': [1e-5, 1e-2, 1, 1e2, 1e5],
              'gamma': [1e-5, 1e-2, 1, 1e2, 1e5],
             'kernel': ['rbf', 'linear']}
svm = GridSearchCV(SVC(class_weight='balanced', cache_size = 2000), param_grid, n_jobs = 6)

def standaryzowanie(tab):
    tabS = prep.StandardScaler().fit_transform(tab)
    tabN = prep.Normalizer().fit_transform(tab)
    tabSN = prep.Normalizer().fit_transform(tabS)
    tabNS = prep.StandardScaler().fit_transform(tabN)
    return tab, tabS, tabN, tabSN, tabNS


def test(nazwa, tablica, tablica2, tablica3, tablica4, tablica5, svm, listZbiorowIndeksow, procent = 0.75, iloscKolumn = 60):
    wynik1, listaWynikow1 = crossValidationPrzedzialy(svm, (((tablica).T[:iloscKolumn]).T), listZbiorowIndeksow, procent)
    wynik2, listaWynikow2 = crossValidationPrzedzialy(svm, (((tablica2).T[:iloscKolumn]).T), listZbiorowIndeksow, procent)
    wynik3, listaWynikow3 = crossValidationPrzedzialy(svm, (((tablica3).T[:iloscKolumn]).T), listZbiorowIndeksow, procent)
    wynik4, listaWynikow4 = crossValidationPrzedzialy(svm, (((tablica4).T[:iloscKolumn]).T), listZbiorowIndeksow, procent) 
    wynik5, listaWynikow5 = crossValidationPrzedzialy(svm, (((tablica5).T[:iloscKolumn]).T), listZbiorowIndeksow, procent) 
    print('Czyste    S       N      S->N    N->S')
    for i in range(10):
        print("{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}    - {5}, tydzień {6}".format(listaWynikow1[i], listaWynikow2[i], listaWynikow3[i], listaWynikow4[i], listaWynikow5[i], zwrocDzien(i), int(i/5) + 1))
    print("Wyniki średnie:")
    print("{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}".format(wynik1, wynik2, wynik3, wynik4, wynik5))


In [9]:
listaDni = np.array([2, 3, 4, 5, 6, 9, 10, 11, 12 ,13])
indeksyDni = []
for dzien in listaDni:
    indeksyDnia = []
    for indeks in oIndeksy:
        if (X[indeks].dzien == dzien):
            indeksyDnia.append(indeks)
    indeksyDni.append(np.array(indeksyDnia))

In [28]:
tab, tabS, tabN, tabSN, tabNS = standaryzowanie(tablicaTickowPo1)
for procent in (np.arange(0.1, 1, 0.05)):
    svm = SVC(class_weight='balanced', gamma = 1e-4, kernel = 'rbf', C = 1, tol = 1e-5, cache_size=8000)
    print("procent danych uczących: ", procent,  "SVM: gamma:", svm.gamma, ", kernel:", svm.kernel, ", C:", svm.C, ", tol:", svm.tol)
    test("imbalance", tab, tabS, tabN, tabSN, tabNS, svm, indeksyDni, procent = procent, iloscKolumn= 120)
    print()

procent danych uczących:  0.1 SVM: gamma: 0.0001 , kernel: rbf , C: 1 , tol: 1e-05
Czyste    S       N      S->N    N->S
0.4924  0.4823  0.4924  0.4722  0.4975    - poniedziałek, tydzień 1
0.5318  0.5273  0.5318  0.5318  0.5295    - wtorek, tydzień 1
0.5023  0.5208  0.5069  0.5208  0.5255    - środa, tydzień 1
0.4602  0.4627  0.5518  0.4554  0.4819    - czwartek, tydzień 1
0.5233  0.4628  0.4744  0.4698  0.4767    - piątek, tydzień 1
0.5254  0.5109  0.5061  0.5133  0.5206    - poniedziałek, tydzień 2
0.4662  0.5188  0.4862  0.5213  0.4687    - wtorek, tydzień 2
0.5120  0.5096  0.5120  0.5096  0.5120    - środa, tydzień 2
0.4915  0.5230  0.5375  0.5351  0.5303    - czwartek, tydzień 2
0.4693  0.5307  0.5307  0.5307  0.5307    - piątek, tydzień 2
Wyniki średnie:
0.4974  0.5049  0.5130  0.5060  0.5073

procent danych uczących:  0.15 SVM: gamma: 0.0001 , kernel: rbf , C: 1 , tol: 1e-05
Czyste    S       N      S->N    N->S
0.4893  0.4733  0.4893  0.4733  0.4679    - poniedziałek, tydzień 1

In [29]:
tab, tabS, tabN, tabSN, tabNS = standaryzowanie(tablicaTickowRosnaca)
for procent in (np.arange(0.1, 1, 0.05)):
    svm = SVC(class_weight='balanced', gamma = 1e-4, kernel = 'rbf', C = 1, tol = 1e-5, cache_size=8000)
    print("procent danych uczących: ", procent,  "SVM: gamma:", svm.gamma, ", kernel:", svm.kernel, ", C:", svm.C, ", tol:", svm.tol)
    test("imbalance", tab, tabS, tabN, tabSN, tabNS, svm, indeksyDni, procent = procent, iloscKolumn= 120)
    print()

procent danych uczących:  0.1 SVM: gamma: 0.0001 , kernel: rbf , C: 1 , tol: 1e-05
Czyste    S       N      S->N    N->S
0.4924  0.4798  0.4747  0.4747  0.4823    - poniedziałek, tydzień 1
0.5318  0.4818  0.4682  0.4818  0.4886    - wtorek, tydzień 1
0.5023  0.5347  0.5000  0.5185  0.5046    - środa, tydzień 1
0.4602  0.4867  0.5349  0.4723  0.4699    - czwartek, tydzień 1
0.5233  0.4907  0.4907  0.5372  0.4721    - piątek, tydzień 1
0.5254  0.4746  0.4407  0.4746  0.4746    - poniedziałek, tydzień 2
0.4662  0.5038  0.5263  0.5213  0.5238    - wtorek, tydzień 2
0.5120  0.5048  0.5120  0.5120  0.4952    - środa, tydzień 2
0.4915  0.5254  0.5206  0.5327  0.5061    - czwartek, tydzień 2
0.4693  0.5012  0.4791  0.5258  0.4619    - piątek, tydzień 2
Wyniki średnie:
0.4974  0.4984  0.4947  0.5051  0.4879

procent danych uczących:  0.15 SVM: gamma: 0.0001 , kernel: rbf , C: 1 , tol: 1e-05
Czyste    S       N      S->N    N->S
0.4893  0.4786  0.4786  0.4652  0.4813    - poniedziałek, tydzień 1

In [30]:
tab, tabS, tabN, tabSN, tabNS = standaryzowanie(tablicaCalekPo1)
for procent in (np.arange(0.1, 1, 0.05)):
    svm = SVC(class_weight='balanced', gamma = 1e-4, kernel = 'rbf', C = 1, tol = 1e-5, cache_size=8000)
    print("procent danych uczących: ", procent,  "SVM: gamma:", svm.gamma, ", kernel:", svm.kernel, ", C:", svm.C, ", tol:", svm.tol)
    test("imbalance", tab, tabS, tabN, tabSN, tabNS, svm, indeksyDni, procent = procent, iloscKolumn= 120)
    print()

procent danych uczących:  0.1 SVM: gamma: 0.0001 , kernel: rbf , C: 1 , tol: 1e-05
Czyste    S       N      S->N    N->S
0.4924  0.4924  0.4874  0.4899  0.4924    - poniedziałek, tydzień 1
0.5318  0.5318  0.5318  0.5250  0.5318    - wtorek, tydzień 1
0.5231  0.5046  0.5231  0.5000  0.5046    - środa, tydzień 1
0.4627  0.4627  0.4627  0.4651  0.4627    - czwartek, tydzień 1
0.4767  0.4767  0.4767  0.4767  0.4767    - piątek, tydzień 1
0.5230  0.5278  0.5206  0.5230  0.5230    - poniedziałek, tydzień 2
0.5113  0.5113  0.5063  0.5138  0.5113    - wtorek, tydzień 2
0.5120  0.5120  0.5120  0.5120  0.5120    - środa, tydzień 2
0.5206  0.5157  0.5230  0.5303  0.5278    - czwartek, tydzień 2
0.4840  0.5307  0.4767  0.5307  0.5307    - piątek, tydzień 2
Wyniki średnie:
0.5038  0.5066  0.5020  0.5066  0.5073

procent danych uczących:  0.15 SVM: gamma: 0.0001 , kernel: rbf , C: 1 , tol: 1e-05
Czyste    S       N      S->N    N->S
0.4706  0.4706  0.5027  0.4679  0.4706    - poniedziałek, tydzień 1

In [31]:
tab, tabS, tabN, tabSN, tabNS = standaryzowanie(tablicaCalekRosnaca)
for procent in (np.arange(0.1, 1, 0.05)):
    svm = SVC(class_weight='balanced', gamma = 1e-4, kernel = 'rbf', C = 1, tol = 1e-5, cache_size=8000)
    print("procent danych uczących: ", procent,  "SVM: gamma:", svm.gamma, ", kernel:", svm.kernel, ", C:", svm.C, ", tol:", svm.tol)
    test("imbalance", tab, tabS, tabN, tabSN, tabNS, svm, indeksyDni, procent = procent, iloscKolumn= 120)
    print()

procent danych uczących:  0.1 SVM: gamma: 0.0001 , kernel: rbf , C: 1 , tol: 1e-05
Czyste    S       N      S->N    N->S
0.4924  0.4924  0.4924  0.4949  0.4924    - poniedziałek, tydzień 1
0.4773  0.4659  0.4773  0.5477  0.4705    - wtorek, tydzień 1
0.4931  0.4861  0.4954  0.4838  0.4861    - środa, tydzień 1
0.4602  0.4602  0.4602  0.4602  0.4602    - czwartek, tydzień 1
0.4721  0.4721  0.4698  0.4512  0.4674    - piątek, tydzień 1
0.4915  0.4891  0.4722  0.4770  0.4794    - poniedziałek, tydzień 2
0.5338  0.5489  0.5363  0.5439  0.5439    - wtorek, tydzień 2
0.5120  0.5168  0.5240  0.5024  0.5192    - środa, tydzień 2
0.5109  0.5133  0.5109  0.5036  0.5109    - czwartek, tydzień 2
0.5086  0.4988  0.5086  0.4914  0.4889    - piątek, tydzień 2
Wyniki średnie:
0.4952  0.4944  0.4947  0.4956  0.4919

procent danych uczących:  0.15 SVM: gamma: 0.0001 , kernel: rbf , C: 1 , tol: 1e-05
Czyste    S       N      S->N    N->S
0.4973  0.4893  0.4893  0.5027  0.4893    - poniedziałek, tydzień 1

In [27]:
def wypisz(TAB):
    tab, tabS, tabN, tabSN, tabNS = standaryzowanie(TAB)
    gammas = [1e-3, 1, 1e3]
    kernels = ['rbf']
    Cs = [1e-3, 1, 1e3]
    tols = [1e-3, 1, 1e3]

    procent = 0.7
    print("procent =", procent)
    print('Czyste  S     N     S->N    N->S')
    for gamma in gammas:
        for C in Cs:
            for tol in tols:
                for kernel in kernels:
                    svm = SVC(class_weight='balanced', gamma = gamma, kernel = kernel, C = C, tol = tol)
                    wynik1, wynik2, wynik3, wynik4, wynik5 = test2(oIndeksy, "imbalance", tab, tabS, tabN, tabSN, tabNS, svm, ilosc = 4, procent = procent, iloscKolumn = 80)
                    odpowiedz = "{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}".format(wynik1, wynik2, wynik3, wynik4, wynik5)
                    if (wynik1 > 0.52 or wynik2 > 0.52 or wynik3 > 0.52 or wynik4 > 0.52 or wynik5 > 0.52):
                        odpowiedz = "\x1b[31m\"{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}\"\x1b[0m".format(wynik1, wynik2, wynik3, wynik4, wynik5)

                    print(odpowiedz, "- dla: g:", gamma, "k:", kernel, "C:", C, "tol:", tol)  

In [32]:
wypisz(tablicaTickowPo1)

procent = 0.7
Czyste  S     N     S->N    N->S
0.4977  0.5081  0.5113  0.5126  0.5117 - dla: g: 0.001 k: rbf C: 0.001 tol: 0.001
0.4977  0.5081  0.5113  0.5126  0.5117 - dla: g: 0.001 k: rbf C: 0.001 tol: 1
0.5023  0.5023  0.5023  0.5023  0.5023 - dla: g: 0.001 k: rbf C: 0.001 tol: 1000.0
"0.4977  0.5203  0.5113  0.5126  0.5108" - dla: g: 0.001 k: rbf C: 1 tol: 0.001
0.4977  0.5068  0.5113  0.5126  0.5153 - dla: g: 0.001 k: rbf C: 1 tol: 1
0.5023  0.5023  0.5023  0.5023  0.5023 - dla: g: 0.001 k: rbf C: 1 tol: 1000.0
0.4977  0.4968  0.5140  0.5068  0.5027 - dla: g: 0.001 k: rbf C: 1000.0 tol: 0.001
0.4977  0.4946  0.5018  0.5000  0.5050 - dla: g: 0.001 k: rbf C: 1000.0 tol: 1
0.5023  0.5023  0.5023  0.5023  0.5023 - dla: g: 0.001 k: rbf C: 1000.0 tol: 1000.0
0.4977  0.5050  0.5050  0.5144  0.5099 - dla: g: 1 k: rbf C: 0.001 tol: 0.001
0.4977  0.5050  0.5050  0.5144  0.5099 - dla: g: 1 k: rbf C: 0.001 tol: 1
0.5023  0.5023  0.5023  0.5023  0.5023 - dla: g: 1 k: rbf C: 0.001 tol: 1000.0


In [33]:
wypisz(tablicaTickowRosnaca)

procent = 0.7
Czyste  S     N     S->N    N->S
0.4977  0.4928  0.4941  0.4887  0.4838 - dla: g: 0.001 k: rbf C: 0.001 tol: 0.001
0.4977  0.4928  0.4941  0.4887  0.4838 - dla: g: 0.001 k: rbf C: 0.001 tol: 1
0.5023  0.5023  0.5023  0.5023  0.5023 - dla: g: 0.001 k: rbf C: 0.001 tol: 1000.0
0.4977  0.4955  0.4941  0.4887  0.4914 - dla: g: 0.001 k: rbf C: 1 tol: 0.001
0.4977  0.4896  0.4941  0.4887  0.4946 - dla: g: 0.001 k: rbf C: 1 tol: 1
0.5023  0.5023  0.5023  0.5023  0.5023 - dla: g: 0.001 k: rbf C: 1 tol: 1000.0
0.4977  0.4905  0.5027  0.5036  0.5050 - dla: g: 0.001 k: rbf C: 1000.0 tol: 0.001
0.4977  0.4905  0.4968  0.5009  0.5072 - dla: g: 0.001 k: rbf C: 1000.0 tol: 1
0.5023  0.5023  0.5023  0.5023  0.5023 - dla: g: 0.001 k: rbf C: 1000.0 tol: 1000.0
0.4977  0.5108  0.4865  0.4860  0.4977 - dla: g: 1 k: rbf C: 0.001 tol: 0.001
0.4977  0.5108  0.4865  0.4860  0.4977 - dla: g: 1 k: rbf C: 0.001 tol: 1
0.5023  0.5023  0.5023  0.5023  0.5023 - dla: g: 1 k: rbf C: 0.001 tol: 1000.0
0.

In [34]:
wypisz(tablicaCalekPo1)

procent = 0.7
Czyste  S     N     S->N    N->S
0.5086  0.4986  0.5050  0.5000  0.5014 - dla: g: 0.001 k: rbf C: 0.001 tol: 0.001
0.5086  0.4986  0.5050  0.5000  0.5014 - dla: g: 0.001 k: rbf C: 0.001 tol: 1
0.5023  0.5023  0.5023  0.5023  0.5023 - dla: g: 0.001 k: rbf C: 0.001 tol: 1000.0
0.5086  0.4968  0.5050  0.5000  0.5000 - dla: g: 0.001 k: rbf C: 1 tol: 0.001
0.5086  0.4977  0.5050  0.5000  0.4991 - dla: g: 0.001 k: rbf C: 1 tol: 1
0.5023  0.5023  0.5023  0.5023  0.5023 - dla: g: 0.001 k: rbf C: 1 tol: 1000.0
0.5086  0.5077  0.5050  0.5005  0.5122 - dla: g: 0.001 k: rbf C: 1000.0 tol: 0.001
0.5086  0.5185  0.5009  0.5000  0.5032 - dla: g: 0.001 k: rbf C: 1000.0 tol: 1
0.5023  0.5023  0.5023  0.5023  0.5023 - dla: g: 0.001 k: rbf C: 1000.0 tol: 1000.0
0.5077  0.5041  0.5068  0.4982  0.5018 - dla: g: 1 k: rbf C: 0.001 tol: 0.001
0.5077  0.5041  0.5068  0.4982  0.5018 - dla: g: 1 k: rbf C: 0.001 tol: 1
0.5023  0.5023  0.5023  0.5023  0.5023 - dla: g: 1 k: rbf C: 0.001 tol: 1000.0
0.

In [35]:
wypisz(tablicaCalekRosnaca)

procent = 0.7
Czyste  S     N     S->N    N->S
0.4991  0.5050  0.5045  0.5018  0.5054 - dla: g: 0.001 k: rbf C: 0.001 tol: 0.001
0.4991  0.5050  0.5045  0.5018  0.5054 - dla: g: 0.001 k: rbf C: 0.001 tol: 1
0.5023  0.5023  0.5023  0.5023  0.5023 - dla: g: 0.001 k: rbf C: 0.001 tol: 1000.0
0.4991  0.4892  0.5045  0.5018  0.4905 - dla: g: 0.001 k: rbf C: 1 tol: 0.001
0.4991  0.4856  0.5045  0.5018  0.4941 - dla: g: 0.001 k: rbf C: 1 tol: 1
0.5023  0.5023  0.5023  0.5023  0.5023 - dla: g: 0.001 k: rbf C: 1 tol: 1000.0
0.4991  0.5018  0.4937  0.5014  0.4959 - dla: g: 0.001 k: rbf C: 1000.0 tol: 0.001
0.4991  0.4964  0.4973  0.4842  0.5018 - dla: g: 0.001 k: rbf C: 1000.0 tol: 1
0.5023  0.5023  0.5023  0.5023  0.5023 - dla: g: 0.001 k: rbf C: 1000.0 tol: 1000.0
0.4995  0.4860  0.4883  0.4955  0.4905 - dla: g: 1 k: rbf C: 0.001 tol: 0.001
0.4995  0.4860  0.4883  0.4955  0.4905 - dla: g: 1 k: rbf C: 0.001 tol: 1
0.5023  0.5023  0.5023  0.5023  0.5023 - dla: g: 1 k: rbf C: 0.001 tol: 1000.0
0.

In [10]:
# testowania dla danego dnia jednego tygodnia uczac sie na tym samym dniu w innym tygodniu

def crossValidationPrzedzialyPary(clf, data, listaIndeksow):
    dataClass = zmiana
    err = 0
    i = 0
    listaWynikow = []
    for indeksy in listaIndeksow:
        trainMask = indeksy[0]
        testMask = indeksy[1]
        X_train = data[trainMask]
        y_train = dataClass[trainMask]
        X_test = data[testMask]
        y_test = dataClass[testMask]  
        clf = clf.fit(X_train, y_train)
        y_train = y_train.reshape((y_train.shape[0], 1))
        wynikDecFun = clf.decision_function(X_train).reshape(y_train.shape)
        najDecFun = najlepszeDecFun(y_train, wynikDecFun)        
        wynik = testujDecisionFunction(X_test, y_test, najDecFun, clf)
        listaWynikow.append(wynik)
        err += wynik
        i += 1
    return float(err) / len(listaIndeksow), listaWynikow


imbalanceS = prep.StandardScaler().fit_transform(imbalance)
imbalanceN = prep.Normalizer().fit_transform(imbalance)
imbalanceSN = prep.Normalizer().fit_transform(imbalanceS)
imbalanceNS = prep.StandardScaler().fit_transform(imbalanceN)


def test(nazwa, tablica, tablica2, tablica3, tablica4, tablica5, svm, listZbiorowIndeksow, procent = 0.75, iloscKolumn = 60):
    wynik1, listaWynikow1 = crossValidationPrzedzialyPary(svm, (((tablica).T[:iloscKolumn]).T), listZbiorowIndeksow)
    wynik2, listaWynikow2 = crossValidationPrzedzialyPary(svm, (((tablica2).T[:iloscKolumn]).T), listZbiorowIndeksow)
    wynik3, listaWynikow3 = crossValidationPrzedzialyPary(svm, (((tablica3).T[:iloscKolumn]).T), listZbiorowIndeksow)
    wynik4, listaWynikow4 = crossValidationPrzedzialyPary(svm, (((tablica4).T[:iloscKolumn]).T), listZbiorowIndeksow) 
    wynik5, listaWynikow5 = crossValidationPrzedzialyPary(svm, (((tablica5).T[:iloscKolumn]).T), listZbiorowIndeksow) 
    print('Czyste    S       N      S->N    N->S')
    for i in range(5):
        print("{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}    - {5}".format(listaWynikow1[i], listaWynikow2[i], listaWynikow3[i], listaWynikow4[i], listaWynikow5[i], zwrocDzien(i)))
    print("Wyniki średnie:")
    print("{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}".format(wynik1, wynik2, wynik3, wynik4, wynik5))

In [11]:
indeksyDniPary = []
i = 0
for dzien in listaDni:
    if (i == 5):
        break
    i += 1
    indeksyDniaUczace = []
    indeksyDniaTestowe =[]
    for indeks in oIndeksy:
        if (X[indeks].dzien == dzien):
            indeksyDniaUczace.append(indeks)
        if (X[indeks].dzien == dzien + 7):
            indeksyDniaTestowe.append(indeks)
    indeksyDniPary.append(np.array([np.array(indeksyDniaUczace), np.array(indeksyDniaTestowe)]))

In [13]:
indeksyDni[2][0].shape, indeksyDni[2][1].shape

((), (), array([ 2,  3,  4,  5,  6,  9, 10, 11, 12, 13]))

In [16]:
tab, tabS, tabN, tabSN, tabNS = standaryzowanie(tablicaCalekPo1)
svm = SVC(class_weight='balanced', gamma = 1e-4, kernel = 'rbf', C = 1, tol = 1e-5, cache_size=8000)
print("SVM: gamma:", svm.gamma, ", kernel:", svm.kernel, ", C:", svm.C, ", tol:", svm.tol)
test("imbalance", tab, tabS, tabN, tabSN, tabNS, svm, indeksyDniPary, iloscKolumn= 120)

SVM: gamma: 0.0001 , kernel: rbf , C: 1 , tol: 1e-05
Czyste    S       N      S->N    N->S
0.4782  0.4716  0.4672  0.5262  0.5262    - poniedziałek
0.4628  0.4289  0.4628  0.4424  0.5463    - wtorek
0.5195  0.5195  0.5195  0.5195  0.5195    - środa
0.4869  0.5109  0.5131  0.4956  0.5197    - czwartek
0.4912  0.4956  0.4889  0.5044  0.4978    - piątek
Wyniki średnie:
0.4877  0.4853  0.4903  0.4976  0.5219


In [17]:
tab, tabS, tabN, tabSN, tabNS = standaryzowanie(tablicaCalekRosnaca)
svm = SVC(class_weight='balanced', gamma = 1e-4, kernel = 'rbf', C = 1, tol = 1e-5, cache_size=8000)
print("SVM: gamma:", svm.gamma, ", kernel:", svm.kernel, ", C:", svm.C, ", tol:", svm.tol)
test("imbalance", tab, tabS, tabN, tabSN, tabNS, svm, indeksyDniPary, iloscKolumn= 120)

SVM: gamma: 0.0001 , kernel: rbf , C: 1 , tol: 1e-05
Czyste    S       N      S->N    N->S
0.5262  0.5284  0.5262  0.5262  0.5284    - poniedziałek
0.4605  0.5395  0.4605  0.5305  0.5418    - wtorek
0.5195  0.5195  0.5195  0.5195  0.5195    - środa
0.5131  0.5044  0.5131  0.5087  0.5044    - czwartek
0.4735  0.4735  0.4735  0.4712  0.4712    - piątek
Wyniki średnie:
0.4985  0.5130  0.4985  0.5112  0.5130


In [60]:
tab, tabS, tabN, tabSN, tabNS = standaryzowanie(tablicaTickowPo1)
svm = SVC(class_weight='balanced', gamma = 1e-4, kernel = 'rbf', C = 1, tol = 1e-5, cache_size=8000)
print("SVM: gamma:", svm.gamma, ", kernel:", svm.kernel, ", C:", svm.C, ", tol:", svm.tol)
test("imbalance", tab, tabS, tabN, tabSN, tabNS, svm, indeksyDniPary, iloscKolumn= 120)

SVM: gamma: 0.0001 , kernel: rbf , C: 1 , tol: 1e-05
Czyste    S       N      S->N    N->S
0.5262  0.4672  0.5197  0.4825  0.5218    - poniedziałek
0.4605  0.4718  0.4786  0.4831  0.4447    - wtorek
0.5195  0.5195  0.5346  0.5195  0.5195    - środa
0.4956  0.4978  0.4934  0.4913  0.5087    - czwartek
0.4735  0.5265  0.5199  0.5177  0.5310    - piątek
Wyniki średnie:
0.4951  0.4966  0.5092  0.4988  0.5051


In [62]:
tab, tabS, tabN, tabSN, tabNS = standaryzowanie(imbalance)
svm = SVC(class_weight='balanced', gamma = 1e-4, kernel = 'rbf', C = 1, tol = 1e-5, cache_size=8000)
print("SVM: gamma:", svm.gamma, ", kernel:", svm.kernel, ", C:", svm.C, ", tol:", svm.tol)
test("imbalance", tab, tabS, tabN, tabSN, tabNS, svm, indeksyDniPary, iloscKolumn= 120)

SVM: gamma: 0.0001 , kernel: rbf , C: 1 , tol: 1e-05
Czyste    S       N      S->N    N->S
0.4782  0.4782  0.5044  0.5087  0.5044    - poniedziałek
0.5440  0.5440  0.5598  0.5598  0.5598    - wtorek
0.5346  0.5346  0.5325  0.5368  0.5325    - środa
0.5262  0.5262  0.5349  0.5349  0.5349    - czwartek
0.5531  0.5531  0.5465  0.5465  0.5465    - piątek
Wyniki średnie:
0.5272  0.5272  0.5356  0.5373  0.5356


In [70]:
tab, tabS, tabN, tabSN, tabNS = standaryzowanie(imbalance)
svm = SVC(class_weight='balanced', gamma = 1e-10, kernel = 'rbf', C = 1, tol = 1e-10, cache_size=8000)
print("SVM: gamma:", svm.gamma, ", kernel:", svm.kernel, ", C:", svm.C, ", tol:", svm.tol)
test("imbalance", tab, tabS, tabN, tabSN, tabNS, svm, indeksyDniPary, iloscKolumn= 120)

SVM: gamma: 1e-10 , kernel: rbf , C: 1 , tol: 1e-10
Czyste    S       N      S->N    N->S
0.4782  0.4782  0.5044  0.5087  0.5044    - poniedziałek
0.5440  0.5440  0.5598  0.5598  0.5598    - wtorek
0.5346  0.5346  0.5325  0.5368  0.5325    - środa
0.5262  0.5262  0.5349  0.5349  0.5349    - czwartek
0.5531  0.5531  0.5465  0.5465  0.5465    - piątek
Wyniki średnie:
0.5272  0.5272  0.5356  0.5373  0.5356


In [64]:
indeksyDniOdwrocone = []
for indeks in indeksyDniPary:
    indeksyDniOdwrocone.append(np.array([indeks[1], indeks[0]]))

In [65]:
tab, tabS, tabN, tabSN, tabNS = standaryzowanie(imbalance)
svm = SVC(class_weight='balanced', gamma = 1e-4, kernel = 'rbf', C = 1, tol = 1e-5, cache_size=8000)
print("SVM: gamma:", svm.gamma, ", kernel:", svm.kernel, ", C:", svm.C, ", tol:", svm.tol)
test("imbalance", tab, tabS, tabN, tabSN, tabNS, svm, indeksyDniOdwrocone, iloscKolumn= 120)

SVM: gamma: 0.0001 , kernel: rbf , C: 1 , tol: 1e-05
Czyste    S       N      S->N    N->S
0.4909  0.4909  0.5318  0.5295  0.5318    - poniedziałek
0.5553  0.5553  0.5471  0.5492  0.5471    - wtorek
0.5365  0.5365  0.5637  0.5637  0.5637    - środa
0.5358  0.5358  0.5423  0.5401  0.5423    - czwartek
0.5283  0.5283  0.5723  0.5723  0.5723    - piątek
Wyniki średnie:
0.5294  0.5294  0.5515  0.5510  0.5515


In [107]:
def test(nazwa, tablica, tablica2, tablica3, tablica4, tablica5, svm, listZbiorowIndeksow, procent = 0.75, iloscKolumn = 60):
    wynik1, listaWynikow1 = crossValidationPrzedzialyPary(svm, (((tablica).T[:iloscKolumn]).T), listZbiorowIndeksow)
    wynik2, listaWynikow2 = crossValidationPrzedzialyPary(svm, (((tablica2).T[:iloscKolumn]).T), listZbiorowIndeksow)
    wynik3, listaWynikow3 = crossValidationPrzedzialyPary(svm, (((tablica3).T[:iloscKolumn]).T), listZbiorowIndeksow)
    wynik4, listaWynikow4 = crossValidationPrzedzialyPary(svm, (((tablica4).T[:iloscKolumn]).T), listZbiorowIndeksow) 
    wynik5, listaWynikow5 = crossValidationPrzedzialyPary(svm, (((tablica5).T[:iloscKolumn]).T), listZbiorowIndeksow) 
    print('Czyste    S       N      S->N    N->S')
    for i in range(len(listZbiorowIndeksow)):
        print("{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}    - ilość dni uczących {5}".format(listaWynikow1[i], listaWynikow2[i], listaWynikow3[i], listaWynikow4[i], listaWynikow5[i], i + 1))

In [108]:
listaDniSkumulowanaPrzedPiatkiem = []
for i in range(1, 10):
    indeksyDniaUczace = indeksyDni[9 - i]
    for j in range(10-i, 9):
        indeksyDniaUczace = np.concatenate((indeksyDniaUczace, indeksyDni[j]))
    listaDniSkumulowanaPrzedPiatkiem.append(np.array([np.array(indeksyDniaUczace), indeksyDni[9]]))

In [109]:
tab, tabS, tabN, tabSN, tabNS = standaryzowanie(imbalance)
svm = SVC(class_weight='balanced', gamma = 1e-4, kernel = 'rbf', C = 1, tol = 1e-5, cache_size=8000)
print("SVM: gamma:", svm.gamma, ", kernel:", svm.kernel, ", C:", svm.C, ", tol:", svm.tol)
test("imbalance", tab, tabS, tabN, tabSN, tabNS, svm, listaDniSkumulowanaPrzedPiatkiem, iloscKolumn= 120)

SVM: gamma: 0.0001 , kernel: rbf , C: 1 , tol: 1e-05
Czyste    S       N      S->N    N->S
0.5553  0.5553  0.5465  0.5465  0.5465    - ilość dni uczących 1
0.5509  0.5509  0.5465  0.5465  0.5465    - ilość dni uczących 2
0.5509  0.5509  0.5465  0.5465  0.5465    - ilość dni uczących 3
0.5531  0.5531  0.5465  0.5465  0.5465    - ilość dni uczących 4
0.5553  0.5553  0.5465  0.5465  0.5465    - ilość dni uczących 5
0.5597  0.5597  0.5465  0.5465  0.5465    - ilość dni uczących 6
0.5597  0.5597  0.5487  0.5465  0.5487    - ilość dni uczących 7
0.5531  0.5531  0.5487  0.5465  0.5487    - ilość dni uczących 8
0.5531  0.5531  0.5487  0.5465  0.5487    - ilość dni uczących 9


In [19]:
listaDniPrzedPiatkiem = []
for i in range(0, 9):
    listaDniPrzedPiatkiem.append(np.array([indeksyDni[i], indeksyDni[9]]))

In [20]:
def test(nazwa, tablica, tablica2, tablica3, tablica4, tablica5, svm, listZbiorowIndeksow, procent = 0.75, iloscKolumn = 60):
    wynik1, listaWynikow1 = crossValidationPrzedzialyPary(svm, (((tablica).T[:iloscKolumn]).T), listZbiorowIndeksow)
    wynik2, listaWynikow2 = crossValidationPrzedzialyPary(svm, (((tablica2).T[:iloscKolumn]).T), listZbiorowIndeksow)
    wynik3, listaWynikow3 = crossValidationPrzedzialyPary(svm, (((tablica3).T[:iloscKolumn]).T), listZbiorowIndeksow)
    wynik4, listaWynikow4 = crossValidationPrzedzialyPary(svm, (((tablica4).T[:iloscKolumn]).T), listZbiorowIndeksow) 
    wynik5, listaWynikow5 = crossValidationPrzedzialyPary(svm, (((tablica5).T[:iloscKolumn]).T), listZbiorowIndeksow) 
    print('Czyste    S       N      S->N    N->S')
    for i in range(len(listZbiorowIndeksow)):
        print("{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}    - dzień uczący przed piątkiem {5}".format(listaWynikow1[i], listaWynikow2[i], listaWynikow3[i], listaWynikow4[i], listaWynikow5[i], i + 1))

In [21]:
tab, tabS, tabN, tabSN, tabNS = standaryzowanie(imbalance)
svm = SVC(class_weight='balanced', gamma = 1e-4, kernel = 'rbf', C = 1, tol = 1e-5, cache_size=8000)
print("SVM: gamma:", svm.gamma, ", kernel:", svm.kernel, ", C:", svm.C, ", tol:", svm.tol)
test("imbalance", tab, tabS, tabN, tabSN, tabNS, svm, listaDniPrzedPiatkiem, iloscKolumn= 120)

SVM: gamma: 0.0001 , kernel: rbf , C: 1 , tol: 1e-05
Czyste    S       N      S->N    N->S
0.5376  0.5376  0.5465  0.5465  0.5465    - dzień uczący przed piątkiem 1
0.5575  0.5575  0.5465  0.5465  0.5465    - dzień uczący przed piątkiem 2
0.5465  0.5465  0.5487  0.5465  0.5487    - dzień uczący przed piątkiem 3
0.5398  0.5398  0.5465  0.5465  0.5465    - dzień uczący przed piątkiem 4
0.5531  0.5531  0.5465  0.5465  0.5465    - dzień uczący przed piątkiem 5
0.4690  0.4690  0.5465  0.5465  0.5465    - dzień uczący przed piątkiem 6
0.5487  0.5487  0.5465  0.5465  0.5465    - dzień uczący przed piątkiem 7
0.5420  0.5420  0.5465  0.5465  0.5465    - dzień uczący przed piątkiem 8
0.5553  0.5553  0.5465  0.5465  0.5465    - dzień uczący przed piątkiem 9


In [116]:
listaDniPrzedPiatkiem[4][0].shape

(477,)